In [ ]:
!pip install -q trl huggingface peft transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch

print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.current_device(), torch.cuda.get_device_name())

CUDA available: True
Device: 0 NVIDIA A100-SXM4-40GB


In [ ]:
MODEL_PATH = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("trl-lib/Capybara", split="train", cache_dir="./hf_cache", download_mode="force_redownload")

def format_conversation(example):
    conversation = ""
    for msg in example["messages"]:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            conversation += f"User: {content.strip()}\n"
        elif role == "assistant":
            conversation += f"Assistant: {content.strip()}\n"
    return {"text": conversation + tokenizer.eos_token}


# Tokenize the formatted text
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,  # or 2048 depending on your model
    )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
dataset = dataset.map(format_conversation)
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names, num_proc=4)
# remove_columns=dataset.column_names is telling the Hugging Face datasets library:
# After applying map(), keep only the new columns I return. Remove the old ones. ❞



train-00000-of-00001.parquet:   0%|          | 0.00/37.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/482k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15806 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/15806 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/15806 [00:00<?, ? examples/s]

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


sft_config = SFTConfig(
    output_dir="Checkpoints",
    per_device_train_batch_size=8,     # ✅ Adjust to your GPU
    gradient_accumulation_steps=2,     # ✅ If small VRAM
    num_train_epochs=3,                # ✅ Choose as needed
    logging_steps=100,
    save_steps=100,
    save_total_limit=2,
    fp16=True,                        # ✅ if using bfloat16
    remove_unused_columns=False,
    dataloader_num_workers = 4,     # ✅ prevents data drop
)


trainer = SFTTrainer(
    model,
    train_dataset=tokenized_dataset,
    args=sft_config,
    peft_config=peft_config
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.413300
200,1.091600
300,1.044900
400,1.037800
500,1.016500
600,1.008800
700,1.011200
800,1.007300
900,1.004800
1000,0.986700


Step,Training Loss
100,1.413300
200,1.091600
300,1.044900
400,1.037800
500,1.016500
600,1.008800
700,1.011200
800,1.007300
900,1.004800
1000,0.986700


TrainOutput(global_step=2964, training_loss=1.0049822957892167, metrics={'train_runtime': 3229.1824, 'train_samples_per_second': 14.684, 'train_steps_per_second': 0.918, 'total_flos': 3.027035428780769e+17, 'train_loss': 1.0049822957892167})

In [ ]:
OUTPUT_MODEL_PATH = f"{MODEL_PATH}-SFT"

# Save only LoRA adapter weights
peft_model = trainer.model
peft_model.save_pretrained(f"{OUTPUT_MODEL_PATH}", safe_serialization=True)

# Save tokenizer (only once is fine)
trainer.tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/tokenizer_config.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/special_tokens_map.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/chat_template.jinja',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/tokenizer.model',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/added_tokens.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-SFT/tokenizer.json')

In [ ]:
print(type(trainer.model))

<class 'peft.peft_model.PeftModelForCausalLM'>


In [ ]:
print(isinstance(trainer.model, PeftModel))

True


In [ ]:
# Optional: If you need to save full model incase
full_model = trainer.model.merge_and_unload()  # merges LoRA into base
full_model.save_pretrained("FULL-MODEL")

In [ ]:
# Push to huggingface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `tinyllama-trl-finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to

In [ ]:
peft_model.push_to_hub("devashish07/tinyllama-finetuning-trl")
trainer.tokenizer.push_to_hub("devashish07/tinyllama-finetuning-trl")

README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/devashish07/tinyllama-finetuning-trl/commit/9e2fb229b1b844eab6de4934c4507c7ff5292a3a', commit_message='Upload tokenizer', commit_description='', oid='9e2fb229b1b844eab6de4934c4507c7ff5292a3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/devashish07/tinyllama-finetuning-trl', endpoint='https://huggingface.co', repo_type='model', repo_id='devashish07/tinyllama-finetuning-trl'), pr_revision=None, pr_num=None)

In [ ]:
# Inferencing
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
LORA_ADAPTER_PATH = "devashish07/tinyllama-finetuning-trl"
TOKENIZER_PATH = "devashish07/tinyllama-finetuning-trl"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)
model.eval()

# Text streamer for real-time printing (optional)
streamer = TextStreamer(tokenizer)

# Prompt
prompt = """User: How do I train a language model with SFT?
Assistant:"""

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )

# Decode
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print("\n\n[Model's Full Output]\n", response)


# Strip prompt to get only the assistant's response
assistant_reply = response.split("Assistant:")[-1].strip()

<s> User: How do I train a language model with SFT?
Assistant: Sure! The SFT (Self-Supervised Feature Transformation) algorithm is a popular method for training language models. It works by using unsupervised learning to learn how to transform text into a set of learnable features, which can then be used to train the language model. Here's a quick overview of how it works:

1. Collect text data: Collect a large dataset of text data that you want to train your language model on. The dataset should include multiple versions of the text data (e.g., different versions for different languages or different contexts).

2. Preprocess the data: Preprocess the text data by removing punctuation, converting all characters to lowercase, and tokenizing the text into a sequence of tokens. This will make it easier for the SFT algorithm to learn the relevant features from the text.

3. Apply SFT: Apply SFT by running the SFT algorithm on the preprocessed data. This algorithm takes as input the text dat

In [ ]:
print("\n🧠 Assistant's Reply:\n", assistant_reply)


🧠 Assistant's Reply:
 Sure, here's an example:

1. Define the training data: We'll be using a corpus of text, which is a collection of sentences, words, and their corresponding labels. We'll use the [Gutenberg corpus](https://www.gutenberg.org/files/2893/2893-h.txt) as the training data. 2. Create a tokenizer: We'll use the [spaCy tokenizer](https://spacy.io/api/tokenizer) to split the text into words and convert them into tokens. We'll also use [NLTK](https://www.nltk.org/) to tokenize the text. 3. Define the training process: We'll define the training process as follows:

- Data preprocessing: Tokenize the text, remove stop words, and split the text into sentences. - Train the model: Use the [GPT-2 model](https://huggingface.co/models/gpt-2) to generate sentences with the training data. - Evaluate the model: Use [Meter](https://huggingface.co/docs/meter/stable) to evaluate the generated sentences on a separate test dataset. 4. Save the model: We'll save the model using the [Huggingf

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextIteratorStreamer
from peft import PeftModel
import torch
import threading

# -------------------------------
# Load Model and Tokenizer
# -------------------------------
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
LORA_ADAPTER_PATH = "devashish07/tinyllama-finetuning-trl"
TOKENIZER_PATH = "devashish07/tinyllama-finetuning-trl"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)

model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)
model = model.to(base_model.device)
model.eval()

# -------------------------------
# Generator-based Streaming Chat
# -------------------------------
def chat_stream(user_input, history):
    history = history or []

    # Reconstruct full prompt
    formatted_prompt = ""
    for message in history:
        role = message["role"]
        content = message["content"]
        if role == "user":
            formatted_prompt += f"User: {content}\n"
        elif role == "assistant":
            formatted_prompt += f"Assistant: {content}\n"
    formatted_prompt += f"User: {user_input}\nAssistant:"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        **inputs,
        "streamer": streamer,
        "max_new_tokens": 500,
        "temperature": 0.7,
        "top_p": 0.9,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
    }

    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    partial_output = ""
    for new_text in streamer:
        partial_output += new_text
        updated_history = history + [
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": partial_output},
        ]
        yield updated_history, updated_history

# -------------------------------
# Gradio Interface
# -------------------------------
with gr.Blocks() as demo:
    gr.Markdown("### 🦙 TinyLlama Chat — Real-time like ChatGPT")

    chatbot = gr.Chatbot(label="Chat", height=500, render_markdown=False, type="messages")
    msg = gr.Textbox(placeholder="Ask something...", show_label=False)
    clear = gr.Button("🧹 Clear")

    state = gr.State([])

    msg.submit(chat_stream, [msg, state], [chatbot, state])
    msg.submit(lambda _: "", msg, msg)  # Clear input box only
    clear.click(lambda: ([], []), None, [chatbot, state])  # Clear everything

demo.launch()


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.l

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55213c60f2d6511711.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
